# Ejercicio 7
Considerando el pagelink_sample.csv, usando una representación de grafos realizar una función genérica que nos permita calcular la centralidad de un contenido cualquiera de la wikipedia mediante random walks. Mostrar el funcionamiento de la implementación con algún contenido incluido en el set de datos.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 56.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=61071e5eb67e67e35162efd5472278f91a18c3989836aba883195b430090336b
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After t

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Workaround problema pagelinks_sample.csv

In [5]:
pagelink_sample_pd = pd.read_csv('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/pagelinks_sample.csv')

In [6]:
pagelink_sample_pd.head()

,pl_from,pl_namespace,pl_title,pl_from_namespace
0,4840492,10,Bandera_cÃ³digo_del_COI_TPE,2
1,950948,0,Esteban_DuÅ¡an,0
2,6418187,0,Insecta,0
3,3436623,0,Pleasure_to_Kill,0
4,4713066,0,DamiÃ¡n_Oliver,0


In [7]:
contents_pd = pd.read_csv('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.csv')

In [8]:
contents_pd.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,NaN,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...


In [9]:
pagelink_reduce_pd = pagelink_sample_pd.loc[pagelink_sample_pd['pl_title'].isin(contents_pd['title'])]

In [10]:
pagelink_reduce_pd.shape

(20572932, 4)

In [11]:
pagelink_reduce_pd.to_parquet('pagelink_reduce.parquet')

In [12]:
del pagelink_sample_pd
del pagelink_reduce_pd
del contents_pd

Fin Workaround

In [13]:
df_pagelink = sqlContext.read.parquet('/content/pagelink_reduce.parquet', header=True, inferSchema=True)
df_pagelink

DataFrame[pl_from: bigint, pl_namespace: bigint, pl_title: string, pl_from_namespace: bigint, __index_level_0__: bigint]

In [14]:
rdd_pagelink = df_pagelink.rdd
rdd_pagelink.take(5)

[Row(pl_from=6418187, pl_namespace=0, pl_title='Insecta', pl_from_namespace=0, __index_level_0__=2),
 Row(pl_from=6571966, pl_namespace=0, pl_title='IMDb', pl_from_namespace=0, __index_level_0__=8),
 Row(pl_from=5461523, pl_namespace=4, pl_title='Vandalismo', pl_from_namespace=3, __index_level_0__=12),
 Row(pl_from=1098340, pl_namespace=0, pl_title='Wikidata', pl_from_namespace=0, __index_level_0__=13),
 Row(pl_from=6224910, pl_namespace=10, pl_title='Referencias', pl_from_namespace=3, __index_level_0__=16)]

In [15]:
df_contents = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.parquet', header=True, inferSchema=True)
df_contents

DataFrame[title: string, id: bigint, namespace: bigint, revision_id: bigint, parent_revision: double, revision_timestamp: string, revisor_username: string, revisor_id: double, revisor_ip: string, revisor_comment: string]

In [16]:
rdd_contents = df_contents.rdd
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [17]:
contentsById = rdd_contents.map(lambda x: (x.title,x.id))
contentsById.take(5)

[('Wikipedia:Artículos solicitados', 5),
 ('Andorra', 7),
 ('Argentina', 10),
 ('Geografía de Andorra', 15),
 ('Demografía de Andorra', 17)]

In [18]:
pagelinkById = rdd_pagelink.map(lambda x: (x.pl_title,x.pl_from))
pagelinkById.take(5)

[('Insecta', 6418187),
 ('IMDb', 6571966),
 ('Vandalismo', 5461523),
 ('Wikidata', 1098340),
 ('Referencias', 6224910)]

In [19]:
grafo_rdd = pagelinkById.join(contentsById).map(lambda x: (x[1][0],x[1][1])).cache()
grafo_rdd.take(10)
# (from,to)

[(5992789, 792610),
 (2171748, 792610),
 (7649514, 792610),
 (370430, 792610),
 (6495197, 792610),
 (8026465, 792610),
 (907985, 792610),
 (6534241, 792610),
 (809476, 792610),
 (8402653, 792610)]

In [20]:
grafo_rdd.count()

20573244

In [21]:
listaAdyacencias = grafo_rdd.groupByKey().map(lambda x: (x[0], list(x[1]))).cache()

In [22]:
listaAdyacencias.takeSample(False,1)[0]

(1223293, [6538])

In [25]:
listaAdyacencias.takeSample(False,1)[0]

(5605538, [675662, 55760, 34715, 181406, 8492, 6858, 235])

In [23]:
import random

In [24]:
def centralidad_aproximada(listaAdyacencias,walk_number=5,walk_length=100):
  cent = {}
  visitados = set()
  for i in range(walk_number):
    v,adyacentes = listaAdyacencias.takeSample(False,1)[0]
    if v not in visitados:
        cent[v] = 1
        visitados.add(v)
    else: cent[v] += 1

    for j in range(walk_length-1):
      while not adyacentes:
          v,adyacentes = listaAdyacencias.takeSample(False,1)[0]
      w = random.choice(adyacentes)
      if w not in visitados:
        cent[w] = 1
        visitados.add(w)
      else: cent[w] += 1
      v = w
  return cent